# Treino RandomForest para classificação de RGs

Este notebook mostra um passo-a-passo para treinar um **RandomForestClassifier** usando o arquivo `features.csv` que você gera com o script de extração de features.

⚠️ **Observação importante**: o arquivo `features.csv` **NÃO** contém rótulos (labels). Você precisa fornecer um arquivo `labels.csv` com duas colunas: `imagem` e `classe`.

Formato de `labels.csv` (exemplo):

```
imagem,classe
rg_001.jpg,novo
rg_002.jpg,antigo
rg_003.jpg,ilegivel
```

O notebook irá:

1. Carregar `features.csv` e `labels.csv` (merge por `imagem`).
2. Converter cores HEX → RGB (valores numéricos).
3. Montar um vetor de características (features).
4. Treinar RandomForest com validação (train/test) e mostrar métricas.
5. Salvar o modelo em `/mnt/data/rf_model.joblib`.

Gerado em: 2025-11-28T15:54:57.578866 UTC


In [ ]:
# Imports e funções auxiliares
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import matplotlib.pyplot as plt

def hex_to_rgb(hexstr):
    """Converte '#rrggbb' -> (r,g,b) inteiros"""
    try:
        h = hexstr.lstrip('#')
        return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    except:
        return (0,0,0)

print('OK: imports carregados')

In [ ]:
# 1) Carregar dados
FEATURES_FILE = './features.csv'   # gerado pelo seu script
LABELS_FILE = './labels.csv'       # você deve criar (imagem,classe)

if not os.path.exists(FEATURES_FILE):
    raise FileNotFoundError(f"Arquivo de features não encontrado: {FEATURES_FILE}")

features = pd.read_csv(FEATURES_FILE)
print('features.shape =', features.shape)
display(features.head())

if not os.path.exists(LABELS_FILE):
    raise FileNotFoundError(f"Arquivo de labels não encontrado: {LABELS_FILE}\n\nCrie um labels.csv com colunas 'imagem' e 'classe'.")

labels = pd.read_csv(LABELS_FILE)
print('labels.shape =', labels.shape)
display(labels.head())

# Merge
df = features.merge(labels, on='imagem', how='inner')
print('merged shape =', df.shape)
display(df.head())

In [ ]:
# 2) Pré-processamento: converter HEX -> RGB e montar feature matrix X e rótulos y
# Esperamos colunas: color1_hex, color2_hex, color3_hex, tem_qrcode, tem_digital, tem_nome, tem_nome_social, tem_nacionalidade

# Converter hex para colunas r,g,b
for i in [1,2,3]:
    col = f'color{i}_hex'
    df[[f'c{i}_r', f'c{i}_g', f'c{i}_b']] = pd.DataFrame(df[col].apply(lambda x: hex_to_rgb(str(x))).tolist(), index=df.index)

# Colunas usadas como features
feature_cols = ['tem_qrcode','tem_digital','tem_nome','tem_nome_social','tem_nacionalidade',
                'c1_r','c1_g','c1_b','c2_r','c2_g','c2_b','c3_r','c3_g','c3_b']

X = df[feature_cols].fillna(0).astype(float)
y = df['classe'].astype(str)

print('X.shape =', X.shape, 'y.shape =', y.shape)
display(X.head())

In [ ]:
# 3) Encode labels em inteiros (se necessário)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y)
print('Classes:', list(le.classes_))
print('Exemplo y_enc:', y_enc[:10])

In [ ]:
# 4) Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42, stratify=y_enc)

print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

In [ ]:
# 5) Treinar RandomForest (baseline)
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Avaliar
y_pred = rf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('\nClassification report:\n', classification_report(y_test, y_pred, target_names=le.classes_))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix:\n', cm)

plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest')
plt.title('Matriz de confusão')
plt.colorbar()
plt.xticks(np.arange(len(le.classes_)), le.classes_, rotation=45)
plt.yticks(np.arange(len(le.classes_)), le.classes_)
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
for (i, j), val in np.ndenumerate(cm):
    plt.text(j, i, int(val), ha='center', va='center', color='white' if val>cm.max()/2 else 'black')
plt.tight_layout()
plt.show()

In [ ]:
# 6) (Opcional) GridSearch para hiperparâmetros - cuidado: pode demorar
params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
gs = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), params, cv=3, scoring='f1_macro', n_jobs=-1)
gs.fit(X_train, y_train)
print('Best params:', gs.best_params_)
best = gs.best_estimator_
y_pred_gs = best.predict(X_test)
print('Accuracy (GS):', accuracy_score(y_test, y_pred_gs))
print('\nClassification report (GS):\n', classification_report(y_test, y_pred_gs, target_names=le.classes_))

In [ ]:
# 7) Salvar o modelo final (usando o baseline rf treinado ou o best do grid)
MODEL_OUT = '/mnt/data/rf_model.joblib'
joblib.dump(rf, MODEL_OUT)
print('Modelo salvo em:', MODEL_OUT)

# Também salvar o label encoder para decodificar previsões
joblib.dump(le, '/mnt/data/label_encoder.joblib')
print('LabelEncoder salvo em: /mnt/data/label_encoder.joblib')

---

## Próximos passos e dicas

- Se tiver poucas amostras por classe, use validação cruzada e coletar mais dados.
- Experimente features adicionais: histogramas, LBP, medidas de nitidez/blur, etc.
- Caso queira usar diretamente imagens (sem features manuais), treine uma CNN com transfer learning (ResNet, EfficientNet).
- Se quiser, eu posso gerar um script Python para rodar esse notebook automaticamente e salvar métricas.

---

**Observação**: depois de executar o notebook, você encontrará o modelo salvo em `/mnt/data/rf_model.joblib` e o encoder em `/mnt/data/label_encoder.joblib`.
